In [ ]:
import numpy as np
import os
import threading
import operator
import pickle
import re
import pandas as pd


start_time = 1190146243
end_time = 1192994591

In [ ]:
def get_lemma_distribution(file_name):
    lemmas=[]
    with open(file_name) as f:
        for line in f:
            url=line.strip().split(' ')[2]
            if url[7:9] != 'en':
                continue
            lemma=url[url.rfind('/')+1:]
            if ':' in lemma:
                continue
            lemmas.append(lemma)
    return lemmas


def count_lemma(dictionary, lemmas):
    for lemma in lemmas:
        if lemma not in dictionary:
            dictionary[lemma] = 1
        else:
            dictionary[lemma] += 1

            
def get_lemma_counter(num, nworkers = 4):
    data_root = "./data/only_lemma/"
    data_files = os.listdir(data_root)
    counter = {}

    threads = [None] * nworkers
    flags = [True] * nworkers
    
    if num > len(data_files):
        num = len(data_files)
    indices = np.random.choice(len(data_files), num)
    
    def target(index, flags, counter, filename):
        count_lemma(counter, get_lemma_distribution(filename))
        flags[index] = True
        
    j = 1
    
    for i in indices:
        index = 0
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        print("Processing file {}".format(j))
        j += 1
        t = threading.Thread(target = target, args = [index, flags, counter, data_root + data_files[i]])
        threads[index] = t
        t.start()
        
    for thread in threads:
        if thread:
            thread.join()
    return counter

def get_url(entry):
    url = "https://en.wikipedia.org/wiki/{}".format(entry)
    return url
    
def sort_by_value(dictionary):
    sorted_dict = sorted(sort_by_value.items(), key=operator.itemgetter(1))
    return sorted_dict

def dump_data(data, output_name):
    with open(output_name, 'wb') as f:
        pickle.dump(data, f)

def load_data(input_name):
    with open(input_name, 'rb') as f:
        data = pickle.load(f)
    return data

def dump_word_frequency(data, output_dir, num = 2000):
    sorted_dict = sorted(counter.items(), key=operator.itemgetter(1), reverse=True)
    with open(output_dir+'hot_{}.csv'.format(num), 'w') as f:
        f.write("entry\tfrequency\n")
        for i in np.random.choice(100000, num):
            item = sorted_dict[i]
            if len(item[0]) == 0:
                continue
            f.write("{}\t{}\n".format(item[0], item[1]))

In [ ]:
def get_lemma_timestamps_from_file(file_name, entry_list, timestamps):
    with open(file_name) as f:
        for line in f:
            timestamp, url=line.strip().split(' ')[1:3]
            if url[7:9] != 'en':
                continue
            lemma=url[url.rfind('/')+1:]
            if lemma in entry_list:
                timestamps[lemma].append(timestamp)
        return timestamps

# get timestamp lists of entry_list

def get_lemma_timestamps(entry_list, nworkers = 10):
    timestamps = {}
    for entry in entry_list:
        timestamps[entry] = []
    data_root = "./data/only_lemma/"
    data_files = os.listdir(data_root)
    
    flags = [True] * nworkers
    threads = [None] * nworkers
    
    def target(index, flags, file_name, entry_list, timestamps):
        get_lemma_timestamps_from_file(file_name, entry_list, timestamps)
        flags[index] = True
    
    i = 0
    index = 0
    for file in data_files:
        while True:
            try:
                index = flags.index(True)
                flags[index] = False
                break
            except:
                continue
        i += 1
        print("Processing file {}".format(i))
        #target(timestamps, entry, data_root + file)
        t = threading.Thread(target = target, args = [index, flags, data_root + file, entry_list, timestamps])
        threads[index] = t
        t.start()
            
    for thread in threads:
        if thread:
            thread.join()
    return timestamps